In [1]:
import sys
sys.path.insert(0, '..')
import alpha_vantage
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from operator import methodcaller
import requests
import datetime
import yaml
import os
import csv
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import traceback
import sys
pd.set_option('display.max_columns', None)

In [6]:
tsdaily_dfs, pe_ratio_dfs = {}, {}
symbols = ['AAPL','AXP','GOOG','META','NVDA','AMZN','PM','SBUX','DIS','MSFT','NFLX','ICE','PLTR','SPOT','NKE','WMT','JNJ','SHOP','TSLA',
           'MA','XOM','V','HD','AMD','PEP','MCD','ABT','IBM','UPS','MO','STZ']

In [7]:
to_remove = ['PM','GOOG','AAPL','MSFT','AMZN']
# to_remove = ['GOOG','AAPL']
for symbol in to_remove:
    if os.path.exists(f'../pe_ratio_dfs/{symbol}.csv'):
        print(f'Removing for {symbol}')
        os.remove(f'../pe_ratio_dfs/{symbol}.csv')
        os.remove(f'../tsdaily_dfs/{symbol}.csv')
    else:
        print(f'Missing {symbol}')

Missing PM
Missing GOOG
Missing AAPL
Missing MSFT
Missing AMZN


In [8]:
# Read CSVs
# for symbol in symbols:
for symbol in [i[:-4] for i in os.listdir('../tsdaily_dfs')]:
    tsdaily_df = pd.read_csv(f'../tsdaily_dfs/{symbol}.csv',index_col=0)
    tsdaily_df.index = pd.to_datetime(tsdaily_df.index)
    tsdaily_dfs[symbol] = tsdaily_df

for symbol in [i[:-4] for i in os.listdir('../pe_ratio_dfs')]:
    pe_ratio_df = pd.read_csv(f'../pe_ratio_dfs/{symbol}.csv',index_col=0)
    pe_ratio_df.index = pd.to_datetime(pe_ratio_df.index)
    for col in pe_ratio_df.columns[:1]:
        pe_ratio_df[col] = pd.to_datetime(pe_ratio_df[col], errors='coerce')
    pe_ratio_dfs[symbol] = pe_ratio_df

In [9]:
for symbol in symbols:
  if symbol not in tsdaily_dfs.keys() or symbol not in pe_ratio_dfs.keys():
    try:
      tsdaily_dfs[symbol], pe_ratio_dfs[symbol] = alpha_vantage.generate_dfs(symbol)
    except KeyError:
      print(f'Failed at {alpha_vantage.get_prev_api_key()}')
      print(traceback.format_exc())
      break

AAPL
108
GOOG
36
AMZN
103
PM
60
MSFT
109


In [10]:
# To CSVs
# for symbol in symbols:
for symbol in tsdaily_dfs.keys():
    tsdaily_dfs[symbol].to_csv(f'../tsdaily_dfs/{symbol}.csv')
for symbol in pe_ratio_dfs.keys():
    pe_ratio_dfs[symbol].to_csv(f'../pe_ratio_dfs/{symbol}.csv')

In [11]:
# Read CSV
overview_df = pd.read_csv('../summary_info/company_overview.csv', index_col=0)

In [12]:
# summary_info = {}
summary_info = overview_df.transpose().to_dict()
for name in tsdaily_dfs.keys():
    # print(name)
    if name in summary_info.keys():
        continue
    info = alpha_vantage.pull_basic_data(name)
    if len(info.keys()) <= 2:
        print(name,info)
        break
    print(f'Successfully added {name}')
    summary_info[name] = alpha_vantage.pull_basic_data(name)
overview_df = pd.DataFrame(summary_info).transpose()

In [9]:
# To CSV
overview_df.to_csv(f'../summary_info/company_overview.csv')

In [9]:
overview_df.head(2)

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,Address,FiscalYearEnd,LatestQuarter,MarketCapitalization,EBITDA,PERatio,PEGRatio,BookValue,DividendPerShare,DividendYield,EPS,RevenuePerShareTTM,ProfitMargin,OperatingMarginTTM,ReturnOnAssetsTTM,ReturnOnEquityTTM,RevenueTTM,GrossProfitTTM,DilutedEPSTTM,QuarterlyEarningsGrowthYOY,QuarterlyRevenueGrowthYOY,AnalystTargetPrice,TrailingPE,ForwardPE,PriceToSalesRatioTTM,PriceToBookRatio,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
V,V,Common Stock,Visa Inc. Class A,Visa Inc. is an American multinational financi...,1403161,NYSE,USD,USA,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC","P.O. BOX 8999, SAN FRANCISCO, CA, US",September,2023-09-30,530196267000,22870000000.0,31.17,1.603,18.31,1.87,0.0081,8.33,15.95,0.529,0.66,0.156,0.465,32653001000,28567000000,8.33,0.217,0.106,283.68,31.17,26.04,16.24,13.69,16.25,23.45,0.951,263.25,207.52,250.92,238.18,1582520000,2023-12-01,2023-11-08
MO,MO,Common Stock,Altria Group,"Altria Group, Inc. (previously known as Philip...",764180,NYSE,USD,USA,MANUFACTURING,CIGARETTES,"6601 WEST BROAD STREET, RICHMOND, VA, US",December,2023-09-30,73434350000,12526000000.0,8.44,6.31,-1.925,3.8,0.0947,4.92,11.53,0.426,0.593,0.218,0.0,20561000000,14183000000,4.92,8.92,-0.025,46.09,8.44,8.31,3.571,27.3,4.707,7.93,0.669,44.96,38.17,41.04,43.61,1768650000,2024-01-10,2023-12-20


In [13]:
overview_df.columns

Index(['Symbol', 'AssetType', 'Name', 'Description', 'CIK', 'Exchange',
       'Currency', 'Country', 'Sector', 'Industry', 'Address', 'FiscalYearEnd',
       'LatestQuarter', 'MarketCapitalization', 'EBITDA', 'PERatio',
       'PEGRatio', 'BookValue', 'DividendPerShare', 'DividendYield', 'EPS',
       'RevenuePerShareTTM', 'ProfitMargin', 'OperatingMarginTTM',
       'ReturnOnAssetsTTM', 'ReturnOnEquityTTM', 'RevenueTTM',
       'GrossProfitTTM', 'DilutedEPSTTM', 'QuarterlyEarningsGrowthYOY',
       'QuarterlyRevenueGrowthYOY', 'AnalystTargetPrice', 'TrailingPE',
       'ForwardPE', 'PriceToSalesRatioTTM', 'PriceToBookRatio', 'EVToRevenue',
       'EVToEBITDA', 'Beta', '52WeekHigh', '52WeekLow', '50DayMovingAverage',
       '200DayMovingAverage', 'SharesOutstanding', 'DividendDate',
       'ExDividendDate'],
      dtype='object')

In [14]:
bollinger_returns= {}
net_change = {}
last_price = {}
for symbol in tsdaily_dfs.keys():
  bollinger_returns[symbol] = alpha_vantage.generate_bollinger_returns(alpha_vantage.create_buy_sell_pairs(tsdaily_dfs, symbol, start_date = '2021-01-01'))
  net_change[symbol] = tsdaily_dfs[symbol].iloc[-1,3] - tsdaily_dfs[symbol].iloc[0,3]
  last_price[symbol] = tsdaily_dfs[symbol].iloc[-1,3]

metrics = pd.DataFrame([bollinger_returns,net_change,last_price],).transpose().rename(columns={0:'bollinger_returns',1:'net_change',2:'last_price'})
metrics['net_over_change'] = metrics['bollinger_returns']/metrics['net_change'].abs()*100
metrics['net_over_price'] = metrics['bollinger_returns']/metrics['last_price']*100
metrics = metrics.sort_values(by='net_over_price',ascending=False)
metrics['Symbol'] = metrics.index
metrics = metrics.iloc[:,[-1]+list(range(len(metrics.columns)-1))]

In [15]:
def generate_bollinger_metrics(tsdaily_dfs, start_date = '2021-01-01', end_date = datetime.date.today()):
    bollinger_returns= {}
    net_change = {}
    last_price = {}
    for symbol in tsdaily_dfs.keys():
        bollinger_returns[symbol] = alpha_vantage.generate_bollinger_returns(
            alpha_vantage.create_buy_sell_pairs(tsdaily_dfs, symbol, start_date = start_date, end_date = end_date))
        net_change[symbol] = tsdaily_dfs[symbol].iloc[-1,3] - tsdaily_dfs[symbol].iloc[0,3]
        last_price[symbol] = tsdaily_dfs[symbol].iloc[-1,3]

    metrics = pd.DataFrame([bollinger_returns,net_change,last_price],).transpose().rename(columns={0:'bollinger_returns',1:'net_change',2:'last_price'})
    metrics['net_over_change'] = metrics['bollinger_returns']/metrics['net_change'].abs()*100
    metrics['net_over_price'] = metrics['bollinger_returns']/metrics['last_price']*100
    metrics = metrics.sort_values(by='net_over_price',ascending=False)
    metrics['Symbol'] = metrics.index
    metrics = metrics.iloc[:,[-1]+list(range(len(metrics.columns)-1))]
    return metrics

In [16]:
alpha_vantage.create_buy_sell_pairs({'GOOG':alpha_vantage.modify_bollinger_sd(tsdaily_dfs['GOOG'],1.5)},'GOOG',start_date = '2023-01-01')

[((Timestamp('2023-02-21 00:00:00'), 92.05),
  (Timestamp('2023-03-15 00:00:00'), 96.55)),
 ((Timestamp('2023-06-21 00:00:00'), 121.26),
  (Timestamp('2023-07-13 00:00:00'), 124.83)),
 ((Timestamp('2023-09-21 00:00:00'), 131.36),
  (Timestamp('2023-10-09 00:00:00'), 139.5)),
 ((Timestamp('2023-10-25 00:00:00'), 126.67),
  (Timestamp('2023-11-22 00:00:00'), 140.02)),
 ((Timestamp('2023-12-04 00:00:00'), 130.63),
  (Timestamp('2023-12-21 00:00:00'), 141.8))]

In [17]:
alpha_vantage.plot_graph(alpha_vantage.modify_bollinger_sd(tsdaily_dfs['GOOG'],1.5), pe_ratio_dfs['GOOG'], 'GOOG', 
                         start_date = '2023-01-01', end_date = datetime.date.today())

In [18]:
symbol = 'PM'
alpha_vantage.plot_graph(alpha_vantage.modify_bollinger_sd(tsdaily_dfs[symbol],2), pe_ratio_dfs[symbol], symbol, 
                         start_date = '2023-01-01', end_date = datetime.date.today())

In [19]:
for symbol in to_remove:
    alpha_vantage.plot_graph(alpha_vantage.modify_bollinger_sd(tsdaily_dfs[symbol],2), pe_ratio_dfs[symbol], symbol, 
                         start_date = '2023-01-01', end_date = datetime.date.today())

In [18]:
pd.merge(generate_bollinger_metrics(tsdaily_dfs,'2023-01-01'),overview_df[['Symbol','Sector','Industry','Beta']],how='left',on='Symbol')

,Symbol,bollinger_returns,net_change,last_price,net_over_change,net_over_price,Sector,Industry,Beta
0,TSLA,104.36,214.5600,238.45,48.639075,43.765989,MANUFACTURING,MOTOR VEHICLES & PASSENGER CAR BODIES,2.316
1,NVDA,188.06,470.6700,494.17,39.955808,38.055730,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,1.642
2,PLTR,5.42,6.7500,16.25,80.296296,33.353846,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.664
3,SHOP,22.98,52.2200,77.90,44.006128,29.499358,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.277
4,AMD,31.58,115.0100,135.32,27.458482,23.337275,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,1.695
5,MSFT,84.51,281.7000,374.07,30.000000,22.592028,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,0.876
6,META,74.59,319.5982,357.83,23.338680,20.845094,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.221
7,SPOT,30.78,38.9000,187.91,79.125964,16.380182,TECHNOLOGY,RADIO BROADCASTING STATIONS,1.629
8,WMT,24.31,101.4000,157.65,23.974359,15.420235,TRADE & SERVICES,RETAIL-VARIETY STORES,0.484
9,NFLX,74.73,475.0400,491.79,15.731307,15.195510,TRADE & SERVICES,SERVICES-VIDEO TAPE RENTAL,1.283


In [19]:
pd.merge(generate_bollinger_metrics(tsdaily_dfs,'2022-01-01','2023-01-01'),overview_df[['Symbol','Sector','Industry','Beta']],how='left',on='Symbol')

,Symbol,bollinger_returns,net_change,last_price,net_over_change,net_over_price,Sector,Industry,Beta
0,GOOG,314.82,-421.0700,137.39,74.766666,229.143315,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.054
1,XOM,16.56,28.4700,103.22,58.166491,16.043402,ENERGY & TRANSPORTATION,PETROLEUM REFINING,0.96
2,IBM,24.22,63.3500,160.10,38.232044,15.128045,TECHNOLOGY,COMPUTER & OFFICE EQUIPMENT,0.764
3,PEP,17.19,139.3900,172.95,12.332305,9.939289,MANUFACTURING,BEVERAGES,0.537
4,MA,40.81,372.7700,418.77,10.947769,9.745206,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",1.093
5,UPS,11.93,89.2900,157.54,13.360959,7.572680,ENERGY & TRANSPORTATION,TRUCKING & COURIER SERVICES (NO AIR),1.055
6,V,14.24,201.4800,257.98,7.067699,5.519808,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",0.951
7,ABT,2.59,69.2100,109.52,3.742234,2.364865,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,0.725
8,JNJ,2.84,51.6400,156.74,5.499613,1.811918,LIFE SCIENCES,PHARMACEUTICAL PREPARATIONS,0.531
9,MCD,2.24,252.7000,294.39,0.886427,0.760895,TRADE & SERVICES,RETAIL-EATING PLACES,0.708


In [20]:
pd.merge(generate_bollinger_metrics(tsdaily_dfs,'2021-01-01','2022-01-01'),overview_df[['Symbol','Sector','Industry','Beta']],how='left',on='Symbol')

,Symbol,bollinger_returns,net_change,last_price,net_over_change,net_over_price,Sector,Industry,Beta
0,SHOP,400.35,52.2200,77.90,766.660283,513.928113,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.277
1,AMZN,429.91,84.2100,153.34,510.521316,280.363897,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,1.163
2,GOOG,259.25,-421.0700,137.39,61.569335,188.696412,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",1.054
3,PLTR,9.53,6.7500,16.25,141.185185,58.646154,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.664
4,DIS,25.80,64.6900,90.38,39.882517,28.546139,TRADE & SERVICES,SERVICES-MISCELLANEOUS AMUSEMENT & RECREATION,1.352
5,NFLX,136.04,475.0400,491.79,28.637588,27.662214,TRADE & SERVICES,SERVICES-VIDEO TAPE RENTAL,1.283
6,SPOT,48.38,38.9000,187.91,124.370180,25.746368,TECHNOLOGY,RADIO BROADCASTING STATIONS,1.629
7,SBUX,21.53,68.2300,95.29,31.555034,22.594186,TRADE & SERVICES,RETAIL-EATING & DRINKING PLACES,0.984
8,MCD,66.01,252.7000,294.39,26.121884,22.422637,TRADE & SERVICES,RETAIL-EATING PLACES,0.708
9,NKE,21.07,52.2600,108.57,40.317643,19.406834,MANUFACTURING,RUBBER & PLASTICS FOOTWEAR,1.12


In [22]:
pd.merge(metrics,overview_df[['Symbol','Sector','Industry','Beta']],how='left',on='Symbol')

,Symbol,bollinger_returns,net_change,last_price,net_over_change,net_over_price,Sector,Industry,Beta
0,GOOG,477.40,-421.0700,137.39,113.377823,347.477982,NaN,NaN,NaN
1,MA,179.02,372.7700,418.77,48.024251,42.749003,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",1.093
2,XOM,30.60,28.4700,103.22,107.481560,29.645418,ENERGY & TRANSPORTATION,PETROLEUM REFINING,0.96
3,PLTR,4.25,6.7500,16.25,62.962963,26.153846,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,2.664
4,MSFT,95.89,281.7000,374.07,34.039759,25.634240,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,0.876
5,V,62.78,201.4800,257.98,31.159420,24.335220,TRADE & SERVICES,"SERVICES-BUSINESS SERVICES, NEC",0.951
6,MCD,71.06,252.7000,294.39,28.120301,24.138048,TRADE & SERVICES,RETAIL-EATING PLACES,0.708
7,PEP,35.13,139.3900,172.95,25.202669,20.312229,MANUFACTURING,BEVERAGES,0.537
8,AXP,36.17,38.0600,187.06,95.034157,19.336042,FINANCE,FINANCE SERVICES,1.216
9,AAPL,33.18,103.5600,181.18,32.039397,18.313280,NaN,NaN,NaN
